# Tokenization
Given that we are essentially looking for relationships between words when creating something like a large language model (LLM), we have to prepare the data in such a way that is conducive to modeling purposes. It unfortunately is very inefficient to use the raw words with no alteration. Imagine all the conjugations of any English verb. For example, consider the word "drop". When used as a verb, "drop" can manifest as any of the following:

- drop
- dropped
- dropping

To further complicate the matter, "drop" can also be used as a noun (as in a water droplet or rain drop).

So how do we get around this? Prior to training an LLM, we convert the input into new representations referred to as **tokens**. Generally speaking, a token represents a 3-5 character word (or part of a longer word); however, tokens can also represent other things including emojis, punctuation, and more.

In recent times, many have converged on a singular strategy for tokenization, and that is called **Byte Pair Encoding (BPE)**. At a high level, BPE seeks to chunk words down into byte pairs, and if there's a particular byte pair that is showing up more often than others, we iterate back through the full vocabulary and update anywhere we saw that popular byte pair with an entirely new token. We keep doing this over and over until we're basically happy with the results.

*Note: One thing I personally am struggling to understand is what the ideal number of iterations (aka merges) to do. This is something I'll have to come back and update later.*

For example, let's say the byte pair reprseenting the letters "th" show up quite often. We will then go back through and update each entry in the vocabulary that has "th" with another arbitrary value, like the letter "Z". (This is just an example. Don't worry too much about precise correctness for now.) As you can imagine, representing a two-character string and a single-character string is more efficient!

In a happy coincidence, I began working on this notebook at the same time that AI master [Andrej Karpathy](https://karpathy.ai) also started work to explain this same concept. This notebook will be largely based upon [his work in this GitHub repo](https://github.com/karpathy/minbpe/tree/master).

## Notebook Setup
Throughout this notebook, we will largely make use of standard Python code. As a simple test, we will be following Andrej's lead by making use of [the same example found on this Wikipedia page](https://en.wikipedia.org/wiki/Byte_pair_encoding). Namely, this example takes the following sample input text:

```
sample_text = 'aaabdaaabac'
```

After three "runs" (aka merges) of the BPE algorithm, the expected outbook should be:

```
expected_output = 'XdXac'
```

The output decomposes into the following sources:

```
X = ZY
Y = ab
Z = aa
```

To ensure that everything is working as expected, we will revisit this sample throughout the notebook.

In [1]:
# Setting the sample text and expected output
sample_text = 'aaabdaaabac'
expected_output = 'XdXac'

## Basic BPE Tokenizer
In this section, we'll start building out the basic BPE tokenizer using standard Python code. This will serve as a basis for sections later on. To keep things simple, we'll take things one step at a time with a robust explanation to ensure you maintain a full intuition.

### UTF-8 Byte Encoding
So far, the examples we have used so far have used strings in their more "natural" form, but this is not actually what we're going to use for the final tokenizer. Remember, words are NOT the only things we tokenize! We also tokenize punctuation, emojis, and more.

What we're going to do is to first convert our input text into their **raw byte values**. Remember, your computer's processor basically operates on calculating a bunch of ones and zeroes, so when you press the letter "a" on your keyboard, your computer is understanding it as something it can work with. The most common standard for managing this is known as the **Unicode Transformation Format - 8-bit**, more commonly known as **UTF-8**. Let's go ahead and demonstrate an example in the cell below.

In [8]:
# Printing the UTF-8 version of my name
name_bytes = list('David Hundley'.encode('utf-8'))
print(f'Here is my name (David Hundley) as UTF-8 encoded bytes:\n{name_bytes}')

# Printing the UTF-8 version of an emoji
emoji_bytes = list('🥳'.encode('utf-8'))
print(f'\nHere is what the 🥳 emoji looks like in byte form:\n{emoji_bytes}')

Here is my name (David Hundley) as UTF-8 encoded bytes:
[68, 97, 118, 105, 100, 32, 72, 117, 110, 100, 108, 101, 121]

Here is what the 🥳 emoji looks like in byte form:
[240, 159, 165, 179]


Notice that when we encode just characters as we do in my name, we get a single integer byte value for each individual letter in my name. However, when we feed through a single emoji, that emoji is represented by a collection of 4 separate integer byte values.

Let's go ahead and apply this same UTF-8 encoding to our sample input.

In [11]:
# Converting the sample to a list of UTF-8 encoded bytes
sample_text_bytes = list(sample_text.encode('utf-8'))
print(f'Sample text bytes:\n{sample_text_bytes}')

Sample text bytes:
[97, 97, 97, 98, 100, 97, 97, 97, 98, 97, 99]


## Instantiating the Vocabulary
Because we are converting things into a bytes format, we can instantiate an initial vocabulary based solely on the number of bytes, which is 256. This will be our baseline as we go forward; however, be aware that this will alter each time we run the BPE algorithm.

In [15]:
# Instantiating the vocabulary with 256 entries, each representing a single byte
vocab = {idx: bytes([idx]) for idx in range(256)}

# Checking out the first few entries
print(f'The first few entries of our new vocab:\n{list(vocab.items())[:5]}')

The first few entries of our new vocab:
[(0, b'\x00'), (1, b'\x01'), (2, b'\x02'), (3, b'\x03'), (4, b'\x04')]


## Finding the Most Popular Byte Pair
As we touched on before, the BPE algorithm is all about finding the most popular byte pairs of tokens and condensing them down into an arbitrary representation.

Before going forward, let's talk briefly about why this arbitrary representation is totally fine. It might surprise you, but LLMs really know nothing about the semantic meanings of words! In other words, an LLM has no idea what pizza is outside of an encoded numerical representation of the characters in that word.

[I wrote a whole blog post about this](https://medium.com/towards-data-science/an-intuition-for-how-models-like-chatgpt-work-c7f01616bd6d), but in a nutshell, the reason that we can build LLMs is because **language maintains a relatively consistent sequencing**. In the English language for example, most of our sentences follow a typical subject-verb structure.

**When we train an LLM, we are basically trying to tease out those patterns in this sequencing so that we can later emulate in a probibilistically correct way.** We won't cover the transformer architecture here, but basically, it doesn't matter how we're representing the words. So long as the LLM is still able to measure the probability of various sequences during its training, the word "pizza" could be "asoijdfoiwencwe". 

Granted, that silly pizza example is highly inefficient since it's expanding the number of characters, so bringing things back on around, we can keep running this BPE algorithm to make the tokenization as efficient as possible.

We're now going to iterate over our sample input text (in byte form) in a "sliding window" fashion... TO BE CONTINUED!